In [ ]:
CREATE OR REPLACE FILE FORMAT docx_format
  TYPE = 'BINARY';

In [ ]:
SELECT *
FROM TABLE(PARSE_DOCUMENT(
    LOCATION => '@Avalanche_Stage',
    FILE_FORMAT => 'docx_format'
));

In [ ]:
CREATE OR REPLACE TABLE parsed_reviews AS
SELECT *
FROM TABLE(PARSE_DOCUMENT(
    LOCATION => '@Avalanche_Stage',
    FILE_FORMAT => 'docx_format'
));

In [ ]:
from docx import Document
from snowflake.snowpark.session import Session
import tempfile

session = Session.builder.get_or_create()

# List files from stage
file_list = session.sql("LIST @Avalanche_Stage").collect()

data_rows = []

for file in file_list:
    file_name = file['name']
    stage_path = f"@Avalanche_Stage/{file_name}"

    with tempfile.NamedTemporaryFile(delete=False, suffix=".docx") as tmp_file:
        session.file.get(stage_path, tmp_file.name)
        doc = Document(tmp_file.name)
        text = "\n".join([p.text for p in doc.paragraphs])
        data_rows.append((file_name, text))

# Create Snowpark DataFrame and write to table
df = session.create_dataframe(data_rows, schema=["filename", "review_text"])
df.write.mode("overwrite").save_as_table("parsed_reviews")